# Exploratory Notebook

# Business Understanding

## Business Context
Our company is entering the competitive movie production industry by establishing a new movie studio. With major corporations increasingly investing in original video content, we need to strategically position ourselves to maximize our chances of box office success.

## Primary Stakeholder
**Head of New Movie Studio** - The key decision-maker who will use our analysis to guide the studio's content creation strategy and investment decisions.

## Key Business Questions

### 1. **What genres are most profitable?**
   - Which movie genres consistently generate the highest box office returns?
   - Are there emerging genres showing strong growth potential?

### 2. **What budget ranges optimize ROI?**
   - What is the relationship between production budget and box office performance?
   - What budget sweet spot maximizes return on investment?

### 3. **When should we release our films?**
   - Which months/seasons show the strongest box office performance?
   - How can we avoid oversaturated release periods?

## Business Objectives
- **Primary Goal:** Identify the most profitable film characteristics to guide our initial movie productions
- **Success Metrics:** Maximize box office revenue and return on investment
- **Risk Mitigation:** Minimize financial losses by avoiding poorly-performing film categories

## Data Requirements & Analysis Considerations

### Dataset Specifications Needed:
1. **What movie data do we have access to?** (Box office numbers, budgets, genres, release dates, ratings, etc.)
2. **What time period does our data cover?** (Recent years only, or historical data?)
3. **Is it domestic box office, international, or worldwide gross?**

### Business Context Clarifications:
4. **What size studio are we targeting?** (Independent/small budget vs. major studio competitor?)
5. **Are there any genre restrictions?** (Family-friendly only, or open to all ratings?)
6. **Geographic focus?** (Domestic US market, international, or global?)

### Analysis Scope Considerations:
7. **Should we consider streaming/digital revenue or just theatrical box office?**
8. **Do we want to analyze competitor studios' strategies?**
9. **Should seasonal trends consider COVID-19 impact on recent years?**

## Success Metrics Framework
**Primary Metrics:**
- **Gross Revenue:** Total box office performance
- **Return on Investment (ROI):** (Revenue - Budget) / Budget × 100
- **Profit Margins:** Net profit as percentage of revenue
- **Budget Efficiency:** Revenue per dollar invested

**Secondary Metrics:**
- **Market Share:** Performance relative to industry
- **Risk Assessment:** Variance in returns by category
- **Audience Reach:** Ticket sales and demographic penetration

## Recommended Visualizations Strategy

### 1. **Genre Analysis**
- **Horizontal Bar Chart:** Average revenue by genre (easy comparison)
- **Box Plot:** Revenue distribution by genre (shows variability and outliers)
- **Scatter Plot:** Budget vs. Revenue by genre (profitability insights)

### 2. **Budget Optimization**
- **Scatter Plot with Trend Line:** Budget vs. ROI (sweet spot identification)
- **Histogram:** Distribution of profitable budget ranges
- **Bubble Chart:** Budget vs. Revenue (bubble size = profit margin)

### 3. **Release Timing**
- **Line Graph:** Monthly box office performance over time
- **Heatmap:** Revenue by month and genre combination
- **Seasonal Bar Chart:** Quarterly performance comparison

### 4. **Risk vs. Reward Analysis**
- **Risk-Return Scatter Plot:** Average return vs. standard deviation by category
- **Waterfall Chart:** Success rate breakdown by film characteristics

## Expected Deliverables
Three concrete, data-driven recommendations that the Head of Movie Studio can immediately implement in the studio's content strategy and production planning.

# Data Understanding

`bom.movie_gross.csv.gz` dataset from Box Office Mojo contains the revenue data for various films. It contains the following five columns:

- **title**: movie title
- **studio**: producing studio
- **domestic_gross**: domestic gross earnings in USD
- **foreign_gross**: international gross earnings
- **year**: year film was released

`im` database from IMDb (Internet Movie Database) contains information about the movies and those who worked on them. It contains eight tables with the following fields:
- `directors` table contains the directors. It contains the following two fields:
    - **movie_id**: unique movie ID (as per IMDb)
    - **person_id**: director's ID (as per IMDb)
- `known_for` table contains the notable works for each person. It contains the following two fields:
    - **person_id**: unique person ID (as per IMDb)
    - **movie_id**: unique movie ID (as per IMDb)
- `movie_akas` table contains the alternative titles of the movies. It contains the following eight fields:
    - **movie_id**: unique movie ID (as per IMDb)
    - **ordering**: order in which movie title appears
    - **title**: alternative title for the movie
    - **region**: country code where movie was shown
    - **language**: language code of the movie
    - **types**: type of alternative title
    - **attributes**: special attributes of the movie
    - **is_original_title**: shows whether alternative title is the origianl title (1 = original title, 0 = alternative title)
- `movie_basics` table contains basic information about the movies. It contains the following six fields:
    - **movie_id**: unique identifier for the movie (as per IMDb)
    - **primary_title**: official title of the movie
    - **original_title**: official title of the movie in the original language
    - **start_year**: year when the movie was released
    - **runtime_minutes**: duration of the movie in minutes
    - **genres**: genre(s) of the movie
- `movie_ratings` table contains the ratings movies were given by IMDb. It contains the following three fields:
    - **movie_id**: unique movie ID (as per IMDb)
    - **averagerating**: average IMDb rating
    - **numvotes**: number of votes received
- `persons` table contains records of those involved in the movies. It has the following five fields:
    - **person_id**: unique identifier for the person (as per IMDb)
    - **primary_name**: full name of the person
    - **birth_year**: year of birth
    - **death_year**: year of death (if dead)
    - **primary_profession**: role in the movie
- `principals` table contains the information of the main cast and crew. It has the following five fields:
    - **movie_id**: unique movie ID (as per IMDb)
    - **ordering**: order of person in credits
    - **person_id**: unique person ID (as per IMDb)
    - **category**: role played
    - **job**: specific role
    - **characters**: character name
- `writers` table contains the following two fields:
    - **movie_id**: unique movie ID (as per IMDb)
    - **person_id**: writer's ID (as per IMDb)

`rt.movie_info.tsv.gz` dataset from Rotten Tomatoes contains general information about the movies. It contains the following twelve columns:
- **id**: unique movie ID (as per Rotten Tomatoes)
- **synopsis**: short summary of the movie
- **rating**: Motion Picture Association of America(MPAA) Rating
- **genre**: genre(s) of the movie
- **director**: name of the director(s)
- **writer**: name of the writer(s)
- **theater_date**: date released in theaters
- **dvd_date**: date released in DVD format
- **currency**: currency symbol
- **box_office**: total revenue from box office
- **runtime**: movie duration in minutes
- **studio**: producing studios

`rt.reviews.tsv.gz` dataset from Rotten Tomatoes contains review information about movies. It contains the following eight columns:
- **id**: unique movie ID (as per Rotten Tomatoes)
- **review**: review text
- **rating**: rating given by critic
- **fresh**: categorization of the review (fresh = positive, rotten = negative)
- **critic**: name of the critic
- **top_critic**: highlights if a critic is regarded as a Top Critic by Rotten Tomatoes (1 = top critic, 0 = not a top critic)
- **publisher**: publication name
- **date**: date of the review

`tmdb.movies.csv.gz` from The Movie Database (TMDb) contains movie and rating informations about various movies. It contains the following nine columns:
- **genre_ids**: list of genre(s) for the movie
- **id**: unique movie ID (as per TMDb)
- **original_language**: language code for the movie
- **original_title**: title of the movie in the official language
- **popularity**: popularity score (as per TMDb)
- **relaease_date**: release date in theatres
- **title**: movie title
- **vote_average**: average user rating
- **vote_count**: total votes received

`tn.movie_budgets.csv.gz` from The Numbers dataset contains movie budget and revenue information. It has the following six columns:
- **id**: unique movie ID (as per The Numbers)
- **release_date**: rlease date in theatres
- **movie**: movie title
- **production_budget**: budget used (in USD)
- **domestic_gross**: revenue generatedfrom country of origin (in USD)
- **worldwide_gross**: total revenue generated worldwide (in USD)


In [19]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from random import gauss
from scipy import stats
import seaborn as sns
from sklearn.linear_model import LinearRegression
from mpl_toolkits import mplot3d
import sklearn.metrics as metrics
import statsmodels.api as sm
import sqlite3

In [44]:
#going through SQl
def load_tables(connection):
    table_names = pd.read_sql(
        "SELECT name FROM sqlite_master WHERE type='table';", connection
    )["name"].tolist()
    sql_tables = {
        table_name: pd.read_sql(f"SELECT * FROM {table_name}", connection)
        for table_name in table_names
    }
    return sql_tables

path = '../Data/im.db'
conn = sqlite3.connect(path)

imdb_tables = load_tables(conn)

print(imdb_tables.keys())


dict_keys(['movie_basics', 'directors', 'known_for', 'movie_akas', 'movie_ratings', 'persons', 'principals', 'writers'])


In [45]:
path_1 = ("../Data/bom.movie_gross.csv")
path_2 = ("../Data/rt.movie_info.tsv")
path_3 = ("../Data/rt.reviews.tsv")
path_4 = ("../Data/tmdb.movies.csv")
path_5 = ("../Data/tn.movie_budgets.csv")

In [46]:

bom_movie = pd.read_csv(path_1)
rt_info = pd.read_csv(path_2, sep='\t', encoding='latin1')
rt_reviews = pd.read_csv(path_3, sep='\t', encoding='latin1')
tmdb_df = pd.read_csv(path_4)
tn_budget = pd.read_csv(path_5)

